In [2]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import numpy as np
import copy
import seaborn as sns
import pandas as pd
from scipy import stats
import math

import pickle
from src.analysis.utils.general_utils import get_good_data, get_sig_chans_per_subject, make_sig_electrodes_per_subject_and_roi_dict, make_or_load_subjects_electrodes_to_ROIs_dict
from src.analysis.preproc.make_epoched_data import trial_ieeg_rand_offset


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']
['c:\\Users

In [3]:
LAB_root = None
task = 'GlobalLocal'
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt': # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else: # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data(task, root=LAB_root)

### actually make the baselines so we can compare them later

In [27]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
subjects = ['D0057']
task = 'GlobalLocal'
LAB_root = None
channels = None
pad_length = 0.5
base_times_length = 0.5
outliers = 10
passband = (70,150)
dec_factor = 8

baseline_configs = [
    {
        "name": "Stimulus_100sec_within1-101sec_fixed_experimentStartBase",
        "baseline_event": "experimentStart",
        "base_times": (1, 101),
        "base_times_length": 100, # The total duration of the baseline epoch
        "rand_offset": False,
    },

    # {
    #     "name": "Stimulus_0.5sec_within1-101sec_randoffset_experimentStartBase",
    #     "baseline_event": "experimentStart",
    #     "base_times": (1, 101),
    #     "base_times_length": 0.5, # The total duration of the baseline epoch
    #     "rand_offset": True,
    # },

    {
        "name": "Stimulus_0.5sec_fixed_preStimulusBase",
        "baseline_event": "Stimulus",
        "base_times": (-0.5, 0),
        "base_times_length": 0.5, # The duration of the snippet taken from the window
        "rand_offset": False,
    },

    {
        "name": "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase",
        "baseline_event": "Stimulus",
        "base_times": (-1, 0),
        "base_times_length": 0.5, # The duration of the snippet taken from the window
        "rand_offset": True,
    },

    {
        "name": "Stimulus_1.0sec_fixed_preStimulusBase",
        "baseline_event": "Stimulus",
        "base_times": (-1, 0),
        "base_times_length": 1.0,
        "rand_offset": False,
    },

    {
        "name": "Stimulus_0.2sec_fixed_preStimulusBase",
        "baseline_event": "Stimulus",
        "base_times": (-0.2, 0),
        "base_times_length": 0.2,
        "rand_offset": False,
    },

    # {
    #     "name": "Stimulus_0.2sec_within1sec_randoffset_preStimulusBase",
    #     "baseline_event": "Stimulus",
    #     "base_times": (-1, 0),
    #     "base_times_length": 0.2,
    #     "rand_offset": True,
    # }
]

In [5]:
def get_baseline(inst: mne.io.BaseRaw, baseline_event: str, base_times: tuple[float, float], pad_length: float, base_times_length: float, outliers: int, passband: tuple[float, float], dec_factor: int, rand_offset: bool):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    if rand_offset:
        trials = trial_ieeg_rand_offset(inst, baseline_event, base_times, base_times_length, pad_length, preload=True)
    else:
        adjusted_base_times = [base_times[0] - pad_length, base_times[1] + pad_length]
        trials = trial_ieeg(inst, baseline_event, adjusted_base_times, preload=True)

    outliers_to_nan(trials, outliers=outliers)
    HG_base = gamma.extract(trials, passband=passband, copy=False, n_jobs=1)
    crop_pad(HG_base, f"{pad_length}s")
    HG_base.decimate(dec_factor)

    del inst
    return HG_base

In [6]:
def make_baselines(subjects, task, baseline_configs, LAB_root, pad_length, outliers, passband, dec_factor):
    """
    Creates baselines by iterating through a list of configuration dictionaries.
    
    Parameters:
    subjects: list of subjects to create baselines for
    task: task to create baselines for
    baseline_configs: list of configuration dictionaries for baselines
    LAB_root: root directory of the lab data
    pad_length: length of padding to use for baseline creation
    outliers: number of outliers to use for baseline creation
    passband: passband to use for baseline creation
    dec_factor: decimation factor to use for baseline creation
    """
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    layout = get_data(task, root=LAB_root)

    for sub in subjects:
        good = get_good_data(sub, layout)

        # Loop through the list of configurations
        for config in baseline_configs:
            print(f"--- Creating baseline: {config['name']} for subject {sub} ---")
            
            # Pass parameters from the config dictionary
            HG_base = get_baseline(good, 
                                   baseline_event=config['baseline_event'], 
                                   base_times=config['base_times'],
                                   base_times_length=config['base_times_length'],
                                   rand_offset=config['rand_offset'],
                                   pad_length=pad_length, 
                                   outliers=outliers, 
                                   passband=passband, 
                                   dec_factor=dec_factor)

            # Use the unique config name for the save directory
            save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', config['name'], sub)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            output_name_base = generate_output_name(config, dec_factor, outliers, passband, pad_length)
            
            HG_base.save(os.path.join(save_dir, f"{sub}_{output_name_base}_base-epo.fif"), overwrite=True)

In [7]:
def generate_output_name(config, dec_factor, outliers, passband, pad_length):
    """Generates a standardized, descriptive name from a config dictionary."""
    
    # Use the specific length from the config
    name = config['name']
    
    offset_str = "randoffset_" if config['rand_offset'] else ""
    
    return (f"{name}_decFactor_{dec_factor}_"
            f"outliers_{outliers}_passband_{passband[0]}-{passband[1]}_"
            f"padLength_{pad_length}s")

In [28]:
def chunk_epochs_object(epochs_obj: mne.Epochs, chunk_duration_sec: float, overlap_sec: float = 0.0) -> mne.EpochsArray:
    """
    Manually breaks a single, long MNE Epochs trial into multiple shorter chunks.

    Args:
        epochs_obj (mne.Epochs):
            An MNE Epochs object that should contain only ONE trial.
        chunk_duration_sec (float):
            The desired duration of each new chunk in seconds.
        overlap_sec (float, optional):
            The overlap between consecutive chunks in seconds. Defaults to 0.0.

    Returns:
        mne.EpochsArray:
            A new MNE EpochsArray object where each trial is a chunk from the
            original data. Returns the original object if it already has more
            than one trial.
    """
    # If the object already has multiple trials, do nothing.
    if len(epochs_obj) > 1:
        print(f"Warning: Object already has {len(epochs_obj)} trials. Returning original object without chunking.")
        return epochs_obj

    # --- 1. Extract Data and Parameters ---
    # The [0] gets the data from the first and only trial
    data_2d = epochs_obj.get_data(copy=True)[0]
    sfreq = epochs_obj.info['sfreq']
    n_channels, n_total_samples = data_2d.shape

    # --- 2. Convert Times to Samples ---
    chunk_len_samples = int(chunk_duration_sec * sfreq)
    overlap_samples = int(overlap_sec * sfreq)
    step_size_samples = chunk_len_samples - overlap_samples

    if n_total_samples < chunk_len_samples:
        raise ValueError(f"The trial is too short ({n_total_samples / sfreq:.2f}s) to create a chunk of {chunk_duration_sec:.2f}s.")

    # --- 3. Loop and Create Chunks ---
    chunks_list = []
    start_idx = 0
    while start_idx + chunk_len_samples <= n_total_samples:
        end_idx = start_idx + chunk_len_samples
        chunk = data_2d[:, start_idx:end_idx]
        chunks_list.append(chunk)
        start_idx += step_size_samples

    if not chunks_list:
        raise ValueError("Could not extract any chunks from the data.")

    # --- 4. Create a New MNE EpochsArray from the Chunks ---
    # Stack the list of 2D chunks into a 3D numpy array (trials, channels, time)
    final_data_array = np.array(chunks_list)

    # Create a new info object based on the original
    new_info = epochs_obj.info.copy()

    # Create the new Epochs object. The time axis for each chunk starts at 0.
    new_chunked_epochs = mne.EpochsArray(final_data_array, info=new_info, tmin=0)

    print(f"Successfully created {len(new_chunked_epochs)} chunks of {chunk_duration_sec}s each.")
    return new_chunked_epochs

In [8]:
# --- Call the new function ---
make_baselines(subjects, task, baseline_configs, LAB_root, pad_length, outliers, passband, dec_factor)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

100%|██████████| 449/449 [02:11<00:00,  3.42it/s]
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\2499320382.py:16: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=8 parameter will result in a sampling frequency of 256.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(dec_factor)


Overwriting existing file.
Overwriting existing file.
--- Creating baseline: Stimulus_1.0sec_fixed_preStimulusBase for subject D0057 ---
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount241.0/BlockTrialCount17.0/Reac

100%|██████████| 448/448 [03:02<00:00,  2.46it/s]
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\2499320382.py:16: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=8 parameter will result in a sampling frequency of 256.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(dec_factor)


--- Creating baseline: Stimulus_0.2sec_fixed_preStimulusBase for subject D0057 ---
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount241.0/BlockTrialCount17.0/ReactionTime499.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/

100%|██████████| 448/448 [01:22<00:00,  5.46it/s]
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\2499320382.py:16: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=8 parameter will result in a sampling frequency of 256.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(dec_factor)


load in some baselines for testing, and also load in the signal to compare them to

In [29]:
loaded_baselines = {}
loaded_signals = {}

for sub in subjects:
    signal_path = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub, f'{sub}_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1-epo.fif')
    loaded_signals[sub] = mne.read_epochs(signal_path)
    loaded_baselines[sub] = {}

    # Loop through the same configs to load the data you just created
    for config in baseline_configs:
        # Reconstruct the path with our consistent naming scheme
        save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', config['name'], sub)
        output_name_base = generate_output_name(config, dec_factor, outliers, passband, pad_length)
        file_path = os.path.join(save_dir, f"{sub}_{output_name_base}_base-epo.fif")

        if os.path.exists(file_path):
            # Use the unique config name as the key
            loaded_baselines[sub][config['name']] = mne.read_epochs(file_path)
        else:
            print(f"Warning: File not found for {sub}, baseline {config['name']}: {file_path}")

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\Stimulus_100sec_within1-101sec_fixed_experimentStartBase\D0057\D0057_Stimulus_100sec_within1-101sec_fixed_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_base-epo.fif ...
    Found the data of interest:
        t =    1000.00 ...  101000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection item

chunk up the experiment start baseline

In [30]:
# ==============================================================================
# --- POST-PROCESSING: Manually Chunk Single-Trial Baselines ---
# This block takes a pre-loaded baseline that has one long trial (like from
# 'experimentStart') and manually breaks it into smaller, consecutive chunks.
# ==============================================================================

print("\n--- Manually chunking long baselines ---")

# --- 1. Define your new baseline configuration ---
# This is the name and properties for your NEW, chunked baseline
chunked_config = {
    'name': 'Stimulus_0.5sec_within1-101sec_chunked_experimentStartBase',
    'base_times': (0, 0.5),  # The times for each chunk are relative, from 0 to 0.5s
    'base_times_length': 0.5,
    'rand_offset': False,
}

# --- 2. Identify the source baseline you want to chunk ---
# This should be the name of the baseline that has one long trial
source_baseline_name = 'Stimulus_100sec_within1-101sec_fixed_experimentStartBase' # Or whichever 'experimentStart' baseline you want to use

# --- 3. Loop through subjects to process the data ---
for sub in subjects:
    if sub not in loaded_baselines or source_baseline_name not in loaded_baselines[sub]:
        print(f"Source baseline '{source_baseline_name}' not found for subject {sub}. Skipping chunking.")
        continue

    print(f"Chunking baseline for subject: {sub}")

    # Get the original single-trial object
    source_epochs = loaded_baselines[sub][source_baseline_name]

    # --- 4. Call the chunking function ---
    # Let's create 0.5-second chunks with no overlap
    try:
        chunked_epochs = chunk_epochs_object(
            epochs_obj=source_epochs,
            chunk_duration_sec=0.5,
            overlap_sec=0.0
        )

        # --- 5. Add the new, chunked object back to your main data dictionary ---
        # It's stored under the new name defined in your config
        loaded_baselines[sub][chunked_config['name']] = chunked_epochs

    except ValueError as e:
        print(f"Could not chunk data for {sub}: {e}")


# --- 6. Add the new configuration to your main config list ---
# This ensures the plotting loop will find and process your new baseline.
# We check if it's already there to prevent adding it multiple times if you re-run the cell.
if not any(d['name'] == chunked_config['name'] for d in baseline_configs):
    baseline_configs.append(chunked_config)

print("\n--- Finished chunking baselines. Proceeding to plotting. ---")

# ==============================================================================
# Your existing plotting code starts here...
# for sub in subjects:
#     ...
# ==============================================================================


--- Manually chunking long baselines ---
Chunking baseline for subject: D0057
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Successfully created 200 chunks of 0.5s each.

--- Finished chunking baselines. Proceeding to plotting. ---


grab sig elecs

In [31]:
# Now, the keys here should match the short, unique names from your configs.
# This makes the lookup direct and removes the need for fuzzy matching.
sig_chans_per_baseline = {}

for config in baseline_configs:
    # --- Generate all potential filenames first ---
    
    # Create a name with integer-formatted passband
    passband_as_int = (int(passband[0]), int(passband[1]))
    sig_run_name_int = generate_output_name(config, dec_factor, outliers, passband_as_int, pad_length) + "_stat_func_ttest_ind_equal_var_False"
    
    # Create a name with float-formatted passband
    passband_as_float = (float(passband[0]), float(passband[1]))
    sig_run_name_float = generate_output_name(config, dec_factor, outliers, passband_as_float, pad_length) + "_stat_func_ttest_ind_equal_var_False"
    
    # Create a list of unique potential names to try
    potential_names = [sig_run_name_int]
    if sig_run_name_int != sig_run_name_float:
        potential_names.append(sig_run_name_float)

    # --- Attempt to load the data using the potential names ---
    loaded_successfully = False
    for i, sig_run_name in enumerate(potential_names):
        try:
            print(f"Attempting ({i+1}/{len(potential_names)}) to load sig data for '{config['name']}' with name: '{sig_run_name}'")
            sig_chans_per_baseline[config['name']] = get_sig_chans_per_subject(subjects, sig_run_name, task='GlobalLocal', LAB_root=LAB_root)
            print("... Success.")
            loaded_successfully = True
            break # Exit the loop as soon as we find a valid file
            
        except (FileNotFoundError, IOError) as e:
            print(f"... Failed to find file. Error: {e}")
        except Exception as e:
            # Catch any other unexpected errors from get_sig_chans_per_subject
            print(f"... An unexpected error occurred: {e}")

    # If all attempts failed, ensure a key exists with an empty value
    if not loaded_successfully:
        print(f"Warning: Could not load any significance data for config: '{config['name']}'. Assigning empty result.")
        sig_chans_per_baseline[config['name']] = {}

Attempting (1/2) to load sig data for 'Stimulus_100sec_within1-101sec_fixed_experimentStartBase' with name: 'Stimulus_100sec_within1-101sec_fixed_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False'
... Failed to find file. Error: [Errno 2] No such file or directory: 'C:\\Users\\jz421\\Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\\freqFilt\\figs\\D0057\\sig_chans_D0057_Stimulus_100sec_within1-101sec_fixed_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False.json'
Attempting (2/2) to load sig data for 'Stimulus_100sec_within1-101sec_fixed_experimentStartBase' with name: 'Stimulus_100sec_within1-101sec_fixed_experimentStartBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False'
... Failed to find file. Error: [Errno 2] No such file or directory: 'C:\\Users\\jz421\\Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\

now let's play around with the loaded data

let's look at lpfc elecs, avg activity during signal period

In [32]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, _ = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, {})



Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!
For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, 

all baselines vs signal

In [33]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # ### FIX: Import mpatches to create legend handles
from scipy.stats import gaussian_kde

# Assume the following variables are pre-loaded and defined:
# subjects, loaded_signals, layout, baseline_configs, loaded_baselines, sig_chans_per_baseline

# --- Main Loop ---
# Loop through each subject
for sub in subjects:
    signal = loaded_signals[sub]
    print(f"--- Processing Subject: {sub} ---")

    # Define a single save directory for the multi-baseline comparison plots
    save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # First, find the set of channels that are common to the signal AND ALL baseline files
    common_channels = set(signal.ch_names)
    for config in baseline_configs: # Use the new config structure
        baseline_name = config['name']
        if sub in loaded_baselines and baseline_name in loaded_baselines[sub]:
            base = loaded_baselines[sub][baseline_name]
            common_channels.intersection_update(base.ch_names)
    
    # Sort the channels for consistent ordering
    common_channels = sorted(list(common_channels))
    
    print(f"Found {len(common_channels)} channels common to signal and all baselines. Generating plots...")

    # --- Grid Plotting Setup ---
    n_channels = len(common_channels)
    if n_channels == 0:
        print("No common channels found across all conditions, skipping subject.")
        continue
        
    rows = 6
    cols = 10
    plots_per_fig = rows * cols
    n_figs = math.ceil(n_channels / plots_per_fig)

    # ### FIX: Create a unified color map for all data types (Signal + Baselines)
    # This ensures a consistent color for each condition across all plot types (KDE, vline, asterisk).
    all_types = ['Signal (0-1.5s)'] + [c['name'] for c in baseline_configs]
    palette_base = ["#E41A1C"] + sns.color_palette("viridis_r", len(baseline_configs))
    color_map = {name: color for name, color in zip(all_types, palette_base)}

    # --- Figure Generation Loop ---
    for fig_num in range(n_figs):
        fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4.5))
        
        start_idx = fig_num * plots_per_fig
        end_idx = min(start_idx + plots_per_fig, n_channels)
        channels_for_fig = common_channels[start_idx:end_idx]

        # --- Channel-level Processing and Plotting ---
        for i, ch_name in enumerate(channels_for_fig):
            ax = axes.flat[i]

            # --- 1. Aggregate Time-Averaged Data ---
            dfs_to_concat = []
            single_trial_baseline_data = {}
            asterisks_to_plot = []

            # a) Get signal data
            signal_cropped = signal.copy().crop(tmin=0, tmax=1.5)
            signal_data = signal_cropped.get_data(picks=ch_name).mean(axis=2).flatten()
            signal_data = signal_data[~np.isnan(signal_data)]
            if signal_data.size > 0:
                dfs_to_concat.append(pd.DataFrame({'Activity': signal_data, 'Type': 'Signal (0-1.5s)'}))

            # b) Get data for each baseline
            for j, config in enumerate(baseline_configs):
                baseline_name = config['name']
                
                if baseline_name not in loaded_baselines.get(sub, {}):
                    print(f'Baseline {baseline_name} not found for subject {sub}. Skipping...')
                    continue

                base = loaded_baselines[sub][baseline_name]
                base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
                base_data = base_cropped.get_data(picks=ch_name).mean(axis=2).flatten()
                base_data = base_data[~np.isnan(base_data)]
                
                if ch_name in sig_chans_per_baseline.get(baseline_name, {}).get(sub, []):
                    # ### FIX: Store baseline name instead of color for later lookup in color_map
                    asterisks_to_plot.append({'data': base_data, 'name': baseline_name})

                if base_data.size == 1:
                    single_trial_baseline_data[baseline_name] = base_data[0]
                elif base_data.size > 1:
                    dfs_to_concat.append(pd.DataFrame({'Activity': base_data, 'Type': baseline_name}))

            if not dfs_to_concat and not single_trial_baseline_data:
                ax.set_title(f'{ch_name}\n(No data)', fontsize=8)
                ax.axis('off')
                continue

            df_combined = pd.concat(dfs_to_concat) if dfs_to_concat else pd.DataFrame({'Activity': [], 'Type': []})

            # --- 2. Visualize the Distributions ---
            if not df_combined.empty:
                sns.kdeplot(data=df_combined, x='Activity', hue='Type', 
                            palette=color_map, # ### FIX: Use the unified color_map
                            fill=True, common_norm=False, ax=ax, legend=False)
            
            # --- 3. Plot Single-Trial Baselines as Vertical Lines ---
            for label, value in single_trial_baseline_data.items():
                ax.axvline(x=value, 
                           color=color_map.get(label, 'k'), # ### FIX: Get color from unified color_map
                           linestyle='--', linewidth=2, label=label)

            # --- 4. Plot Asterisks for Significant Channels ---
            if asterisks_to_plot:
                ylim = ax.get_ylim()
                ax.set_ylim(top=ylim[1] * 1.15) # Add extra space for asterisks
                ylim = ax.get_ylim()
                
                # ### FIX: Stagger asterisks vertically to prevent overlap
                y_offset_factor = 0.08 
                
                for k, ast_info in enumerate(asterisks_to_plot):
                    peak_y = ylim[1] * (0.95 - k * y_offset_factor)
                    color = color_map.get(ast_info['name'], 'k') # ### FIX: Get color from map
                    
                    if len(ast_info['data']) == 1:
                        peak_x = ast_info['data'][0]
                    elif len(ast_info['data']) > 1:
                        kde = gaussian_kde(ast_info['data'])
                        x_range = np.linspace(min(ast_info['data']), max(ast_info['data']), 500)
                        kde_vals = kde(x_range)
                        peak_x = x_range[np.argmax(kde_vals)]
                    
                    ax.text(peak_x, peak_y, '*', color=color, 
                            fontsize=24, ha='center', va='center', weight='bold')

            # --- 5. Annotate and Finalize Subplot ---
            ax.set_title(f'Channel: {ch_name}', fontsize=10)
            ax.set_xlabel('Time-Averaged High Gamma Activity', fontsize=8)
            ax.set_ylabel('Density', fontsize=8)
            ax.tick_params(axis='x', labelsize=7, rotation=30)
            ax.tick_params(axis='y', labelsize=7)

            # ### FIX: Create a single, unified legend on the first subplot ###
            if i == 0:
                handles = []
                # Get handles for vertical lines that were actually plotted
                line_handles, line_labels = ax.get_legend_handles_labels()
                handles.extend(line_handles)

                # Create patch handles for KDE plots that were actually plotted
                plotted_kde_types = df_combined['Type'].unique()
                patch_handles = [mpatches.Patch(color=color_map[name], label=name) 
                                 for name in all_types if name in plotted_kde_types]
                
                handles.extend(patch_handles)
                
                # Create a unique list of handles and labels
                # (dict.fromkeys preserves order and removes duplicates)
                unique_labels_map = {h.get_label(): h for h in handles}
                ax.legend(handles=unique_labels_map.values(), labels=unique_labels_map.keys())


        # --- Figure-level Cleanup and Saving ---
        for i in range(len(channels_for_fig), plots_per_fig):
            axes.flat[i].axis('off')

        fig.suptitle(f'Signal vs. All Baselines (Time-Averaged) for Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)
        fig.tight_layout(rect=[0, 0.03, 1, 0.96])
        
        save_name = f"{sub}_multi_baseline_vs_post_stim_signal_comparison_page_{fig_num + 1}.png"
        save_path = os.path.join(save_dir, save_name)
        plt.savefig(save_path, dpi=300)
        print(f"Saved figure to: {save_path}")
        plt.close(fig)

--- Processing Subject: D0057 ---
Found 175 channels common to signal and all baselines. Generating plots...


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (-0.5 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmin is not in time interval. tmin is set to <class 'mne.epochs.EpochsFIF'>.tmin (-0.199219 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (0.496094 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (-0.5 s)
  base_cropped = ba

Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_multi_baseline_vs_post_stim_signal_comparison_page_1.png


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (-0.5 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmin is not in time interval. tmin is set to <class 'mne.epochs.EpochsFIF'>.tmin (-0.199219 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (0.496094 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (-0.5 s)
  base_cropped = ba

Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_multi_baseline_vs_post_stim_signal_comparison_page_2.png


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (-0.5 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmin is not in time interval. tmin is set to <class 'mne.epochs.EpochsFIF'>.tmin (-0.199219 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (0.496094 s)
  base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\432276441.py:87: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsFIF'>.tmax (-0.5 s)
  base_cropped = ba

Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_multi_baseline_vs_post_stim_signal_comparison_page_3.png


make time series plots with mean and stdev across trials for different baselines

In [34]:
# =======================================================================================
# --- REVISED SECTION: Individual Time-Series Comparison Plotting ---
# This section generates separate sets of plots for the signal and each baseline
# condition. For each condition, it plots the mean high-gamma trace over time for
# all common channels across multiple pages.
# =======================================================================================

print("\n--- Starting Individual Time-Series Plotting ---")

# Loop through each subject
for sub in subjects:
    print(f"--- Processing Subject for Individual Time-Series Plots: {sub} ---")

    # --- Create a list of all data objects to plot ---
    # Each item in the list will be a tuple: (string_name, MNE_Epochs_object)
    conditions_to_plot = []

    # 1. Add the Signal data
    conditions_to_plot.append(("Signal", loaded_signals[sub]))

    # 2. Add all available Baseline data objects
    for config in baseline_configs:
        baseline_name = config['name']
        if sub in loaded_baselines and baseline_name in loaded_baselines[sub]:
            conditions_to_plot.append((baseline_name, loaded_baselines[sub][baseline_name]))

    # --- Iterate through each condition (Signal, then each Baseline) ---
    for condition_name, data_object in conditions_to_plot:

        print(f"  -- Generating plots for condition: {condition_name} --")

        # Define a save directory for this specific condition's plots
        save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Determine channels to plot based on the CURRENT data object
        channels_for_this_condition = sorted(data_object.ch_names)

        if not channels_for_this_condition:
            print(f"  -- No channels found for condition {condition_name}, skipping.")
            continue

        # --- Grid Plotting Setup (based on current channels) ---
        rows = 6
        cols = 10
        plots_per_fig = rows * cols
        n_figs = math.ceil(len(channels_for_this_condition) / plots_per_fig)

        # --- Figure Generation Loop (for pagination) ---
        for fig_num in range(n_figs):
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4.5), constrained_layout=True)

            start_idx = fig_num * plots_per_fig
            end_idx = min(start_idx + plots_per_fig, len(channels_for_this_condition))
            channels_for_fig = channels_for_this_condition[start_idx:end_idx]

            # --- Channel-level Processing and Plotting ---
            for i, ch_name in enumerate(channels_for_fig):
                ax = axes.flat[i]

                # Get the data for the current channel
                timeseries_data = data_object.get_data(picks=ch_name, copy=True)
                # Calculate mean and std across epochs (shape is (1, 641))
                mean_2d = np.nanmean(timeseries_data, axis=0)
                std_2d = np.nanstd(timeseries_data, axis=0)

                # --- FIX: Use np.squeeze() to remove the redundant dimension ---
                mean_trace = np.squeeze(mean_2d) # Shape becomes (641,)
                std_trace = np.squeeze(std_2d)   # Shape becomes (641,)

                if i == 1:
                    print(std_trace)

                time_vector = data_object.times

                # Plot the main trace and standard deviation
                ax.plot(time_vector, mean_trace, color="#2c7fb8", linewidth=1.5)
                ax.fill_between(time_vector, mean_trace - std_trace, mean_trace + std_trace, color="#2c7fb8", alpha=0.3)

                # --- MODIFICATION: Add baseline info ONLY to Signal plots ---
                plot_title = f'Channel: {ch_name}' # Default title

                if condition_name == "Signal":
                    # 1. Add black asterisk to TITLE for significance vs. PRE-STIMULUS baseline
                    prestim_key = 'Stimulus_0.5sec_within1sec_randoffset_preStimulusBase'
                    sig_chans_for_sub_prestim = sig_chans_per_baseline.get(prestim_key, {}).get(sub, [])
                    if ch_name in sig_chans_for_sub_prestim:
                        plot_title += ' *'

                    # 2. Add horizontal bar for EXPERIMENT START baseline mean
                    exp_start_key = 'Stimulus_100sec_within1-101sec_experimentStartBase'
                    if sub in loaded_baselines and exp_start_key in loaded_baselines[sub]:
                        base_obj = loaded_baselines[sub][exp_start_key]
                        base_data = base_obj.get_data(picks=ch_name, copy=True)
                        mean_base_activity = np.nanmean(base_data)

                        # Always plot the horizontal line itself
                        ax.axhline(y=mean_base_activity, color='r', linestyle='--', linewidth=2.5)

                        # --- FIX: Only plot the red asterisk if the channel is significant vs. THIS baseline ---
                        sig_chans_for_sub_exp_start = sig_chans_per_baseline.get(exp_start_key, {}).get(sub, [])
                        if ch_name in sig_chans_for_sub_exp_start:
                            # This asterisk only appears if the signal is significantly different from the experiment start baseline
                            ax.text(time_vector[-1], mean_base_activity, '*', color='r', fontsize=24,
                                    ha='center', va='center', weight='bold')

                # --- Finalize Subplot ---
                ax.set_title(plot_title, fontsize=10) # Use the potentially modified title
                ax.set_xlabel('Time (s)', fontsize=8)
                ax.set_ylabel('High Gamma', fontsize=8)
                ax.tick_params(axis='both', labelsize=7)
                ax.axvline(0, color='k', linestyle=':', linewidth=1.5)
                ax.grid(True, which='both', linestyle='--', linewidth=0.5)
                ax.set_xlim(time_vector[0], time_vector[-1])

            # --- Figure-level Cleanup and Saving ---
            for i in range(len(channels_for_fig), plots_per_fig):
                axes.flat[i].axis('off')

            fig.suptitle(f'Time-Series for {condition_name} - Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)

            safe_condition_name = condition_name.replace("/", "_")
            save_name = f"{sub}_{safe_condition_name}_timeseries_page_{fig_num + 1}.png"
            save_path = os.path.join(save_dir, save_name)

            plt.savefig(save_path, dpi=300)
            print(f"    Saved figure to: {save_path}")
            plt.close(fig)


--- Starting Individual Time-Series Plotting ---
--- Processing Subject for Individual Time-Series Plots: D0057 ---
  -- Generating plots for condition: Signal --
[1.24852703e-06 1.23444502e-06 1.22137033e-06 1.21116255e-06
 1.20680194e-06 1.20669099e-06 1.20879297e-06 1.21106552e-06
 1.21362019e-06 1.21616152e-06 1.21850772e-06 1.22199988e-06
 1.22569787e-06 1.22936046e-06 1.23394223e-06 1.24112305e-06
 1.25052158e-06 1.26143294e-06 1.27375816e-06 1.28807474e-06
 1.30591389e-06 1.32739504e-06 1.35098178e-06 1.37269505e-06
 1.38825626e-06 1.39543849e-06 1.39333235e-06 1.38353255e-06
 1.36793975e-06 1.34773727e-06 1.32631474e-06 1.30764626e-06
 1.29384601e-06 1.28756815e-06 1.29123694e-06 1.29974832e-06
 1.30797565e-06 1.31372135e-06 1.31557856e-06 1.31360311e-06
 1.30877176e-06 1.30243953e-06 1.29669502e-06 1.29013434e-06
 1.28287845e-06 1.27883103e-06 1.27989727e-06 1.28652971e-06
 1.29423432e-06 1.30242683e-06 1.30941583e-06 1.31617885e-06
 1.32197722e-06 1.32690955e-06 1.33336328e-

C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\2307932006.py:65: RuntimeWarning: Mean of empty slice
  mean_2d = np.nanmean(timeseries_data, axis=0)
c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_Stimulus_100sec_within1-101sec_fixed_experimentStartBase_timeseries_page_1.png
[0. 0. 0. ... 0. 0. 0.]
    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_Stimulus_100sec_within1-101sec_fixed_experimentStartBase_timeseries_page_2.png
[0. 0. 0. ... 0. 0. 0.]
    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_Stimulus_100sec_within1-101sec_fixed_experimentStartBase_timeseries_page_3.png
  -- Generating plots for condition: Stimulus_0.5sec_fixed_preStimulusBase --
[1.05340362e-06 1.05024404e-06 1.04922352e-06 1.04943717e-06
 1.04936676e-06 1.05095349e-06 1.05576205e-06 1.06186668e-06
 1.07240319e-06 1.08503680e-06 1.10138989e-06 1.11830553e-06
 1.13540128e-06 1.15240499e-06 1.16862496e-06 1.18332773e-06
 1.19695458e-06

make time series plots for the rois in rois_dict

In [35]:
# =======================================================================================
# --- REVISED SECTION 2.0: ROI-Specific Time-Series Comparison Plotting ---
# This section generates plots for specific ROIs. For each ROI, it creates a
# separate set of plots for the signal and each baseline condition, containing only
# the channels belonging to that ROI for that subject.
# =======================================================================================

print("\n--- Starting ROI-Specific Time-Series Plotting ---")

# Define the ROIs to iterate over from your dictionary keys
rois_to_plot = list(rois_dict.keys()) # This will be ['lpfc', 'occ']

# Loop through each subject
for sub in subjects:
    print(f"--- Processing Subject: {sub} ---")

    # === NEW: Loop through each Region of Interest ===
    for roi_name in rois_to_plot:
        print(f"  -- Preparing plots for ROI: {roi_name} --")

        # Get the list of channels for the current subject and ROI
        # The .get() methods prevent errors if a subject/ROI combo doesn't exist
        channels_for_roi = electrodes_per_subject_roi.get(roi_name, {}).get(sub, [])

        if not channels_for_roi:
            print(f"  -- No channels found for Subject {sub} in ROI {roi_name}. Skipping. --")
            continue # Move to the next ROI

        # --- Create a list of all data objects to plot (same as before) ---
        conditions_to_plot = []
        conditions_to_plot.append(("Signal", loaded_signals[sub]))
        for config in baseline_configs:
            baseline_name = config['name']
            if sub in loaded_baselines and baseline_name in loaded_baselines[sub]:
                conditions_to_plot.append((baseline_name, loaded_baselines[sub][baseline_name]))

        # --- Iterate through each condition (Signal, then each Baseline) ---
        for condition_name, data_object in conditions_to_plot:

            print(f"    -- Generating plots for condition: {condition_name} --")

            # === MODIFIED: Define a save directory for this specific subject AND ROI ===
            save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub, roi_name)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            # === MODIFIED: Determine channels to plot by finding the intersection ===
            # This ensures we only try to plot channels that are BOTH in the ROI AND present in the current data file.
            all_channels_in_data = data_object.ch_names
            channels_to_plot = sorted([ch for ch in channels_for_roi if ch in all_channels_in_data])

            if not channels_to_plot:
                print(f"    -- No common channels between ROI '{roi_name}' and condition '{condition_name}'. Skipping. --")
                continue

            # --- Grid Plotting Setup (based on the filtered channel list) ---
            rows = 6
            cols = 10
            plots_per_fig = rows * cols
            n_figs = math.ceil(len(channels_to_plot) / plots_per_fig)

            # --- Figure Generation Loop (for pagination) ---
            for fig_num in range(n_figs):
                fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4.5), constrained_layout=True)

                start_idx = fig_num * plots_per_fig
                end_idx = min(start_idx + plots_per_fig, len(channels_to_plot))
                channels_for_fig = channels_to_plot[start_idx:end_idx]

                # --- Channel-level Processing and Plotting ---
                for i, ch_name in enumerate(channels_for_fig):
                    ax = axes.flat[i]
                    timeseries_data = data_object.get_data(picks=ch_name, copy=True)
                    mean_trace = np.squeeze(np.nanmean(timeseries_data, axis=0))
                    std_trace = np.squeeze(np.nanstd(timeseries_data, axis=0))
                    time_vector = data_object.times

                    ax.plot(time_vector, mean_trace, color="#2c7fb8", linewidth=1.5)
                    ax.fill_between(time_vector, mean_trace - std_trace, mean_trace + std_trace, color="#2c7fb8", alpha=0.3)

                    # --- MODIFICATION: Add baseline info ONLY to Signal plots (same logic) ---
                    plot_title = f'Channel: {ch_name}'
                    if condition_name == "Signal":
                        prestim_key = 'Stimulus_0.5sec_within1sec_randoffset_preStimulusBase'
                        sig_chans_for_sub_prestim = sig_chans_per_baseline.get(prestim_key, {}).get(sub, [])
                        if ch_name in sig_chans_for_sub_prestim:
                            plot_title += ' *'

                        exp_start_key = 'Stimulus_100sec_within1-101sec_experimentStartBase'
                        if sub in loaded_baselines and exp_start_key in loaded_baseline[sub]:
                            base_obj = loaded_baselines[sub][exp_start_key]
                            if ch_name in base_obj.ch_names: # Check if channel exists in baseline object
                                base_data = base_obj.get_data(picks=ch_name, copy=True)
                                mean_base_activity = np.nanmean(base_data)
                                ax.axhline(y=mean_base_activity, color='r', linestyle='--', linewidth=2.5)
                                sig_chans_for_sub_exp_start = sig_chans_per_baseline.get(exp_start_key, {}).get(sub, [])
                                if ch_name in sig_chans_for_sub_exp_start:
                                    ax.text(time_vector[-1], mean_base_activity, '*', color='r', fontsize=24, ha='center', va='center', weight='bold')

                    # --- Finalize Subplot ---
                    ax.set_title(plot_title, fontsize=10)
                    ax.set_xlabel('Time (s)', fontsize=8)
                    ax.set_ylabel('High Gamma', fontsize=8)
                    ax.tick_params(axis='both', labelsize=7)
                    ax.axvline(0, color='k', linestyle=':', linewidth=1.5)
                    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
                    ax.set_xlim(time_vector[0], time_vector[-1])

                # --- Figure-level Cleanup and Saving ---
                for i in range(len(channels_for_fig), plots_per_fig):
                    axes.flat[i].axis('off')

                # === MODIFIED: Update suptitle and save_name for ROI ===
                fig.suptitle(f'ROI: {roi_name.upper()} - Condition: {condition_name} - Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)
                safe_condition_name = condition_name.replace("/", "_")
                save_name = f"{sub}_{roi_name}_{safe_condition_name}_timeseries_page_{fig_num + 1}.png"
                save_path = os.path.join(save_dir, save_name)

                plt.savefig(save_path, dpi=300)
                print(f"      Saved figure to: {save_path}")
                plt.close(fig)


--- Starting ROI-Specific Time-Series Plotting ---
--- Processing Subject: D0057 ---
  -- Preparing plots for ROI: lpfc --
    -- Generating plots for condition: Signal --


NameError: name 'loaded_baseline' is not defined

bar plot for summary statistics

In [36]:
# =======================================================================================
# --- NEW SECTION: Summary Statistic Bar Plots ---
# This section generates two sets of figures for each subject:
# 1. Bar plots comparing the MEAN time-averaged activity for the signal vs. all baselines.
# 2. Bar plots comparing the VARIANCE of the time-averaged activity for the same conditions.
# Each figure contains plots for multiple channels, paginated for readability.
# =======================================================================================

print("\n--- Starting Summary Statistic Bar Plotting ---")

# Loop through each subject
for sub in subjects:
    print(f"--- Processing Subject for Summary Plots: {sub} ---")

    # First, find the set of channels that are common to the signal AND ALL baseline files
    common_channels = set(loaded_signals[sub].ch_names)
    for config in baseline_configs:
        baseline_name = config['name']
        if sub in loaded_baselines and baseline_name in loaded_baselines[sub]:
            base = loaded_baselines[sub][baseline_name]
            common_channels.intersection_update(base.ch_names)
    
    # Sort the channels for consistent ordering
    common_channels = sorted(list(common_channels))
    
    if not common_channels:
        print(f"No common channels found for subject {sub}, skipping summary plots.")
        continue

    print(f"Found {len(common_channels)} common channels. Generating summary plots...")
    
    # --- Pre-calculate all stats for each common channel to avoid redundant computation ---
    all_channels_stats = {}
    for ch_name in common_channels:
        summary_data = []
        
        # Get signal stats (time-averaged across 0-1.5s as requested)
        signal_avg_data = loaded_signals[sub].copy().crop(tmin=0, tmax=1.5).get_data(picks=ch_name).mean(axis=2).flatten()
        summary_data.append({'Type': 'Signal', 'Mean': np.nanmean(signal_avg_data), 'Var': np.nanvar(signal_avg_data)})

        # Get stats for each baseline
        for config in baseline_configs:
            baseline_name = config['name']
            if baseline_name in loaded_baselines.get(sub, {}):
                base = loaded_baselines[sub][baseline_name]
                # Use the actual baseline period for averaging by cropping from None to None
                base_avg_data = base.copy().crop(tmin=None, tmax=None).get_data(picks=ch_name).mean(axis=2).flatten()
                summary_data.append({'Type': baseline_name, 'Mean': np.nanmean(base_avg_data), 'Var': np.nanvar(base_avg_data)})
        
        all_channels_stats[ch_name] = pd.DataFrame(summary_data)

    # --- Plotting Configuration ---
    # We will loop through the stats_to_plot list to create plots for 'Mean' and 'Var'
    stats_to_plot = ['Mean', 'Var']
    
    for stat_type in stats_to_plot:
        
        print(f"  -- Generating {stat_type} comparison plots --")
        
        # Define a save directory for this specific type of statistic
        save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        # --- Grid Plotting Setup ---
        rows = 6
        cols = 10
        plots_per_fig = rows * cols
        n_figs = math.ceil(len(common_channels) / plots_per_fig)

        # --- Figure Generation Loop (for pagination) ---
        for fig_num in range(n_figs):
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4.5), constrained_layout=True)
            
            start_idx = fig_num * plots_per_fig
            end_idx = min(start_idx + plots_per_fig, len(common_channels))
            channels_for_fig = common_channels[start_idx:end_idx]
            
            # --- Channel-level Processing and Plotting ---
            for i, ch_name in enumerate(channels_for_fig):
                ax = axes.flat[i]
                
                df_stats = all_channels_stats[ch_name]

                # Define a color palette
                palette = ["#E41A1C"] + sns.color_palette("viridis_r", len(baseline_configs))

                # Create the bar plot for the current stat_type ('Mean' or 'Var')
                sns.barplot(data=df_stats, x='Type', y=stat_type, ax=ax, palette=palette)

                # --- Finalize Subplot ---
                ax.set_title(f'Channel: {ch_name}', fontsize=10)
                ax.set_ylabel(f'{stat_type} of Activity', fontsize=8)
                ax.set_xlabel(None) # Remove x-label to avoid clutter
                ax.tick_params(axis='y', labelsize=7)
                # Rotate x-axis labels for readability
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
                ax.grid(axis='y', linestyle='--', linewidth=0.5)

            # --- Figure-level Cleanup and Saving ---
            # Hide unused subplots
            for i in range(len(channels_for_fig), plots_per_fig):
                axes.flat[i].axis('off')

            fig.suptitle(f'Comparison of Activity {stat_type} - Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)
            
            # Generate a clean filename and save
            save_name = f"{sub}_summary_stats_{stat_type.lower()}_page_{fig_num + 1}.png"
            save_path = os.path.join(save_dir, save_name)
            
            plt.savefig(save_path, dpi=300)
            print(f"    Saved figure to: {save_path}")
            plt.close(fig)



--- Starting Summary Statistic Bar Plotting ---
--- Processing Subject for Summary Plots: D0057 ---
Found 175 common channels. Generating summary plots...


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:48: RuntimeWarning: Mean of empty slice
  summary_data.append({'Type': baseline_name, 'Mean': np.nanmean(base_avg_data), 'Var': np.nanvar(base_avg_data)})
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:48: RuntimeWarning: Degrees of freedom <= 0 for slice.
  summary_data.append({'Type': baseline_name, 'Mean': np.nanmean(base_avg_data), 'Var': np.nanvar(base_avg_data)})


  -- Generating Mean comparison plots --


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or us

    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_summary_stats_mean_page_1.png


...

    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_summary_stats_var_page_1.png


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or us

    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_summary_stats_var_page_2.png


C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=7)
C:\Users\jz421\AppData\Local\Temp\ipykernel_3120\1294075695.py:97: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or us

    Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_summary_stats_var_page_3.png


trial by trial relationships

In [38]:
# =======================================================================================
# --- NEW SECTION: Pre- vs. Post-Stimulus Correlation Scatter Plots ---
# This section generates scatter plots to investigate the trial-by-trial correlation
# between the mean activity in the pre-stimulus baseline period and the subsequent
# post-stimulus signal period.
# =======================================================================================

print("\n--- Starting Pre- vs. Post-Stimulus Correlation Plotting ---")

# --- Identify the specific pre-stimulus baseline configuration to use ---
# We find the config dictionary dynamically instead of relying on a fixed index
pre_stim_config = None
for config in baseline_configs:
    if "preStimulusBase" in config['name'] and "0.5sec" in config['name']:
        pre_stim_config = config
        break

if pre_stim_config is None:
    print("Could not find the required pre-stimulus baseline configuration. Skipping correlation plots.")
else:
    print(f"Using baseline for correlation: '{pre_stim_config['name']}'")

    # Loop through each subject
    for sub in subjects:
        print(f"--- Processing Subject for Correlation Plots: {sub} ---")

        # --- Data Validation ---
        # Check if the required baseline and signal files are loaded for the subject
        baseline_name = pre_stim_config['name']
        if sub not in loaded_signals or sub not in loaded_baselines or baseline_name not in loaded_baselines[sub]:
            print(f"Missing signal or pre-stimulus baseline data for subject {sub}. Skipping.")
            continue

        signal = loaded_signals[sub]
        pre_stim_base = loaded_baselines[sub][baseline_name]

        # Find channels that are common to this specific signal and baseline pair
        channels_to_plot = sorted(list(set(signal.ch_names).intersection(pre_stim_base.ch_names)))

        if not channels_to_plot:
            print(f"No common channels found between signal and pre-stimulus baseline for {sub}. Skipping.")
            continue
            
        # Ensure trial counts match, which is critical for a paired correlation
        if len(signal.events) != len(pre_stim_base.events):
            print(f"Trial count mismatch for {sub}: Signal has {len(signal.events)} trials, "
                  f"Baseline has {len(pre_stim_base.events)} trials. Skipping correlation plots.")
            continue

        print(f"Found {len(channels_to_plot)} common channels. Generating correlation plots...")
        
        # --- Define Save Directory ---
        save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        # --- Grid Plotting Setup ---
        rows = 6
        cols = 10
        plots_per_fig = rows * cols
        n_figs = math.ceil(len(channels_to_plot) / plots_per_fig)

        # --- Figure Generation Loop (for pagination) ---
        for fig_num in range(n_figs):
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4), constrained_layout=True)
            
            start_idx = fig_num * plots_per_fig
            end_idx = min(start_idx + plots_per_fig, len(channels_to_plot))
            channels_for_fig = channels_to_plot[start_idx:end_idx]
            
            # --- Channel-level Processing and Plotting ---
            for i, ch_name in enumerate(channels_for_fig):
                ax = axes.flat[i]

                # Get trial-wise average for baseline (-0.5 to 0)
                pre_stim_activity = pre_stim_base.get_data(picks=ch_name).mean(axis=2).flatten()

                # Get trial-wise average for signal (0 to 1.5s)
                signal_activity = signal.copy().crop(tmin=0, tmax=1.5).get_data(picks=ch_name).mean(axis=2).flatten()

                # Check for correlation, handle potential NaN values from nanmean
                valid_indices = ~np.isnan(pre_stim_activity) & ~np.isnan(signal_activity)
                if np.sum(valid_indices) < 2: # Need at least 2 points to correlate
                    corr, p_val = np.nan, np.nan
                else:
                    corr, p_val = stats.pearsonr(pre_stim_activity[valid_indices], signal_activity[valid_indices])

                # Create the regression plot
                sns.regplot(x=pre_stim_activity, y=signal_activity, ax=ax,
                            scatter_kws={'s': 10, 'alpha': 0.6},
                            line_kws={'color': 'red', 'linewidth': 2})

                # --- Finalize Subplot ---
                title_str = (f'{ch_name}\n'
                             f'Corr = {corr:.2f}, p = {p_val:.3f}')
                ax.set_title(title_str, fontsize=9)
                ax.set_xlabel('Pre-Stimulus Mean Activity', fontsize=7)
                ax.set_ylabel('Post-Stimulus Mean Activity', fontsize=7)
                ax.tick_params(axis='both', labelsize=7)
                ax.grid(True, linestyle='--', linewidth=0.5)

            # --- Figure-level Cleanup and Saving ---
            # Hide unused subplots
            for i in range(len(channels_for_fig), plots_per_fig):
                axes.flat[i].axis('off')

            fig.suptitle(f'Pre- vs. Post-Stimulus Correlation - Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)
            
            # Generate a clean filename and save
            save_name = f"{sub}_correlation_page_{fig_num + 1}.png"
            save_path = os.path.join(save_dir, save_name)
            
            plt.savefig(save_path, dpi=300)
            print(f"    Saved figure to: {save_path}")
            plt.close(fig)


--- Starting Pre- vs. Post-Stimulus Correlation Plotting ---
Using baseline for correlation: 'Stimulus_0.5sec_fixed_preStimulusBase'
--- Processing Subject for Correlation Plots: D0057 ---
Trial count mismatch for D0057: Signal has 449 trials, Baseline has 448 trials. Skipping correlation plots.


### average across trials and timepoints but not channels, plot avg mean and stdev for each channel

In [20]:
data = HG_base_stimulus.get_data()

mean = np.mean(data, axis=(0,2))
stdev = np.std(data,axis=(0,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base_stimulus.ch_names

# Create a bar plot
plt.figure(figsize=(10, 5))
plt.bar(channel_names, mean, yerr=stdev, align='center', alpha=0.75, ecolor='black', capsize=5)

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.title('Mean and Standard Deviation of Channels')
plt.xticks(rotation=90)  # Rotate the channel names for better readability

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [2.35596596e-05            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
 7.60397970e-06 7.19475317e-06 6.55293008e-06 6.04449749e-06
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan  

KeyboardInterrupt: 

### average across trials, channels, and time points

In [21]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean of experiment start baseline: ", mean)
print("Standard Deviation of experiment start baseline: ", stdev)

Mean of experiment start baseline:  7.11109295788856e-06
Standard Deviation of experiment start baseline:  2.467975522575185e-06


In [22]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean of fixation cross baseline: ", mean)
print("Standard Deviation of fixation cross baseline: ", stdev)

Mean of fixation cross baseline:  6.0587577502376924e-06
Standard Deviation of fixation cross baseline:  4.567779012040867e-06


### average across trials and channels but not timepoints

In [23]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1))
stdev = np.nanstd(data,axis=(0,1))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base.ch_names


# Now, create a plot
plt.figure(figsize=(10, 5))
plt.plot(HG_base.times, mean, label='Mean')
plt.fill_between(HG_base.times, mean - stdev, mean + stdev, alpha=0.2, label='STD')

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.xlabel('Time (s)')
plt.title('Mean and Standard Deviation of timepoints across trials and channels')
plt.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [6.21817376e-06 6.23033093e-06 6.24244690e-06 ... 7.53210802e-06
 7.54389375e-06 7.55552610e-06]
Standard Deviation:  [1.16503499e-06 1.17829252e-06 1.19179487e-06 ... 2.64761902e-06
 2.64209893e-06 2.63672973e-06]


KeyboardInterrupt: 

In [24]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1))
stdev = np.nanstd(data,axis=(0,1))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base_stimulus.ch_names


# Now, create a plot
plt.figure(figsize=(10, 5))
plt.plot(HG_base_stimulus.times, mean, label='Mean')
plt.fill_between(HG_base_stimulus.times, mean - stdev, mean + stdev, alpha=0.2, label='STD')

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.xlabel('Time (s)')
plt.title('Mean and Standard Deviation of timepoints across trials and channels')
plt.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [5.96941022e-06 5.96859328e-06 5.96779262e-06 ... 6.24856718e-06
 6.24944321e-06 6.25032414e-06]
Standard Deviation:  [4.04477745e-06 4.03667350e-06 4.02874892e-06 ... 4.98159151e-06
 4.97979517e-06 4.97793714e-06]


KeyboardInterrupt: 